# TSP USING GENETIC ALGORITHM

In [1]:
import math
from collections import defaultdict
import numpy as np
import random

In [2]:
def preprocessor(nodes,size):
    max_nod_num = max(nodes)
    population = []
    for i in range(size):
        chromosome = []
        while len(chromosome)!=len(nodes):
            random_node = np.random.randint(max_nod_num+1)
            if random_node not in chromosome:
                chromosome.append(random_node)
        population.append(chromosome)
    return population

In [3]:
def fitnessFunction(graph_edges,chromosome):
        total_fitness_cost=0
        i=1
        while i<len(chromosome):
            for temp_edge in graph_edges:
                if chromosome[i-1]==temp_edge[0] and chromosome[i]==temp_edge[1]:
                    total_fitness_cost=total_fitness_cost+temp_edge[2]
            i=i+1
        for temp_edge in graph_edges:
                if chromosome[0]==temp_edge[0] and chromosome[len(chromosome) - 1]==temp_edge[1]:
                    total_fitness_cost=total_fitness_cost+temp_edge[2]
        return total_fitness_cost

In [4]:
def select_top_k_fittest(parent_gen,graph_edges,chromosome_size):
    costs = []
    selected_parent = []
    populationFitness = []
    for i in range(len(parent_gen)):
        costs.append(fitnessFunction(graph_edges,parent_gen[i]))
        populationFitness.append((costs[i],parent_gen[i]))
    populationFitness.sort()
    for i in range(chromosome_size):
        selected_parent.append(populationFitness[i][1])
    return selected_parent,populationFitness[0][0],selected_parent[0]

In [5]:
def Mating(parent1, parent2):
    child = []
    DNA1 = []
    DNA2 = []
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    if geneA < geneB :
        startGene, endGene = geneA, geneB
    else :
        endGene, startGene = geneA, geneB

    for i in range(startGene, endGene):
        DNA1.append(parent1[i])
    
    DNA2 = [item for item in parent2 if item not in DNA1]

    child = DNA1 + DNA2
    return child

def MatePopulation(parents, size):
    children = []
    temp = np.array(parents)
    n_parents = temp.shape[0]
    for i in range(size):
        random_dad = parents[np.random.randint(low =0,high = n_parents - 1)]
        random_mom = parents[np.random.randint(low =0,high = n_parents - 1)]
        children.append(Mating(random_dad,random_mom))
    return children

In [6]:
def mutateIndividual(parent, n_mutations):
    newParent = np.array(parent)
    size1 = newParent.shape[0]
    max_nod_num = max(parent)
    for i in range(n_mutations):
        randomMutation1 = np.random.randint(0,size1)
        randomMutation2 = np.random.randint(0,size1)
        if randomMutation1 == randomMutation2:
            randomMutation2=(randomMutation2+1)%size1
        parent[randomMutation1],parent[randomMutation2]=parent[randomMutation2],parent[randomMutation1]
    return parent


def mutatePopulation(population, n_mutations):
    mutatedPop = []
    for ind in range(0, len(population)):
        mutatedInd = mutateIndividual(population[ind], n_mutations)
        mutatedPop.append(mutatedInd)
    return mutatedPop


In [11]:
class Graph:

    def __init__(self,vertices):
        self.nodes = []
        for i in range(len(vertices)):
            self.nodes.append(vertices[i])
        self.edges = []
        self.successors = defaultdict(list)


    def addEdge(self,u,v,w): 
        for edges in self.edges:
            if u==edges[0] and v==edges[1]:
                print("Edge already exists")
                return
        self.edges.append([u,v,w])
        self.successors[u].append((v, w))


    def total_optimal_cost(self,visited_nodes):
        if len(visited_nodes)<=1:
            return 0
        else:
            total_cost=0
            i=1
            while i<len(visited_nodes):
                for temp_edge in self.edges:
                    if visited_nodes[i-1]==temp_edge[0] and visited_nodes[i]==temp_edge[1]:
                        total_cost=total_cost+temp_edge[2]
                i=i+1
            for temp_edge in self.edges:
                    if visited_nodes[0]==temp_edge[0] and visited_nodes[len(visited_nodes) - 1]==temp_edge[1]:
                        total_cost=total_cost+temp_edge[2]
            return total_cost

    def isdisconnected(self,initial_node):
        is_disconnected = False
        for node in range(len(self.nodes)):
            neighbors = self.successors[node]
            if len(neighbors) < (len(self.nodes)-1):
                is_disconnected = True
                return is_disconnected
        return is_disconnected


    def gen_algo(self,source,generations):
        if self.isdisconnected(source):
            print("Graph is not connected")
            return []
        sizeofPopulation = 20
        parent_gen = preprocessor(self.nodes,sizeofPopulation) 
        overall_costs = []
        overall_routes = []
        for i in range(generations):
            print("Generation number :",i+1,"/",generations)
            chromosome_size = 10
            parent_gen,min_cost,best_route = select_top_k_fittest(parent_gen,self.edges,chromosome_size)
            print("Best route for generation",i+1,":",best_route)
            print("Best cost for generation",i+1,":",min_cost)
            overall_costs.append(min_cost)
            overall_routes.append(best_route)
            parent_gen = MatePopulation(parent_gen,sizeofPopulation)
            n_mutations=1 
            parent_gen = mutatePopulation(parent_gen,n_mutations)
            print()
        minimum = min(overall_costs)
        min_index=-1
        for i in range(len(overall_costs)):
            if minimum == overall_costs[i]:
                min_index = i
        return overall_routes[min_index]


In [12]:
def optimal_path(path,source):
    print("====================================Final Path===================================")
    print("final path:")
    start = path.index(source)
    for i in range(start,len(path)-1):
        print(path[i],"->",path[i+1])
    print(path[len(path)-1],"->",path[0])
    for i in range(0,start):
        print(path[i],"->",path[i+1])

In [26]:
g = Graph([0,1,2,3,4])

g.addEdge(0, 1, 2) 
g.addEdge(1, 0, 2)

g.addEdge(0,2,3)
g.addEdge(2,0,3)

g.addEdge(0, 3, 6) 
g.addEdge(3, 0, 6)

g.addEdge(0, 4, 7) 
g.addEdge(4, 0, 7)

g.addEdge(1, 2, 3) 
g.addEdge(2, 1, 3)

g.addEdge(1, 3, 8) 
g.addEdge(3, 1, 8)
 
g.addEdge(1, 4, 5) 
g.addEdge(4, 1, 5)

g.addEdge(3, 2, 8) 
g.addEdge(2, 3, 8)

g.addEdge(4, 2, 7) 
g.addEdge(2, 4, 7)

g.addEdge(3, 4, 9) 
g.addEdge(4, 3, 9)

generations=100
path = g.gen_algo(0,generations) # executes the algorithm
total_cost = g.total_optimal_cost(path)
if total_cost:
    optimal_path(path,0)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Generation number : 1 / 100
Best route for generation 1 : [0, 1, 4, 3, 2]
Best cost for generation 1 : 27

Generation number : 2 / 100
Best route for generation 2 : [0, 3, 4, 1, 2]
Best cost for generation 2 : 26

Generation number : 3 / 100
Best route for generation 3 : [4, 1, 2, 0, 3]
Best cost for generation 3 : 26

Generation number : 4 / 100
Best route for generation 4 : [0, 3, 4, 1, 2]
Best cost for generation 4 : 26

Generation number : 5 / 100
Best route for generation 5 : [3, 4, 1, 2, 0]
Best cost for generation 5 : 26

Generation number : 6 / 100
Best route for generation 6 : [1, 2, 0, 3, 4]
Best cost for generation 6 : 26

Generation number : 7 / 100
Best route for generation 7 : [2, 0, 3, 4, 1]
Best cost for generation 7 : 26

Generation number : 8 / 100
Best route for generation 8 : [1, 4, 3, 2, 0]
Best cost for generation 8 : 27

Generation number : 9 / 100
Best route for generation 9 : [3, 4, 1, 2, 0]
Best cost for generation 9 : 26

Generation number : 10 / 100
Best rou

In [27]:
g=Graph([0,1,2,3,4,5,6,7])

g.addEdge(0, 1, 10) 
g.addEdge(1, 0, 10)

g.addEdge(0, 2, 15) 
g.addEdge(2, 0, 15)

g.addEdge(0, 3, 20) 
g.addEdge(3, 0, 20)

g.addEdge(0, 4, 19) 
g.addEdge(4, 0, 19)

g.addEdge(0, 5, 13) 
g.addEdge(5, 0, 13)

g.addEdge(0, 6, 28) 
g.addEdge(6, 0, 28)

g.addEdge(0, 7, 29) 
g.addEdge(7, 0, 29)

g.addEdge(1, 2, 35) 
g.addEdge(2, 1, 35)

g.addEdge(1, 3, 25) 
g.addEdge(3, 1, 25)

g.addEdge(1, 4, 29) 
g.addEdge(4, 1, 29)

g.addEdge(1, 5, 37) 
g.addEdge(5, 1, 37)

g.addEdge(1, 6, 18) 
g.addEdge(6, 1, 18)

g.addEdge(1, 7, 21) 
g.addEdge(7, 1, 21)

g.addEdge(2, 3, 8)
g.addEdge(3, 2, 8) 

g.addEdge(2, 4, 11)
g.addEdge(4, 2, 11) 

g.addEdge(2, 5, 15)
g.addEdge(5, 2, 15) 

g.addEdge(2, 6, 17)
g.addEdge(6, 2, 17) 

g.addEdge(2, 7, 22)
g.addEdge(7, 2, 22) 

g.addEdge(3, 4, 14)
g.addEdge(4, 3, 14) 

g.addEdge(3, 5, 9)
g.addEdge(5, 3, 9) 

g.addEdge(3, 6, 23)
g.addEdge(6, 3, 23) 

g.addEdge(3, 7, 26)
g.addEdge(7, 3, 26) 

g.addEdge(4, 5, 27)
g.addEdge(5, 4, 27) 

g.addEdge(4, 6, 31)
g.addEdge(6, 4, 31) 

g.addEdge(4, 7, 32)
g.addEdge(7, 4, 32) 

g.addEdge(5, 6, 38)
g.addEdge(6, 5, 38) 

g.addEdge(5, 7, 33)
g.addEdge(7, 5, 33) 

g.addEdge(6, 7, 39)
g.addEdge(7, 6, 39) 

generations=500
path = g.gen_algo(0,generations) # executes the algorithm
total_cost = g.total_optimal_cost(path)
if total_cost:
    optimal_path(path,0)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Generation number : 1 / 500
Best route for generation 1 : [5, 3, 2, 4, 0, 1, 6, 7]
Best cost for generation 1 : 147

Generation number : 2 / 500
Best route for generation 2 : [5, 0, 1, 6, 3, 4, 2, 7]
Best cost for generation 2 : 144

Generation number : 3 / 500
Best route for generation 3 : [2, 4, 0, 6, 1, 7, 5, 3]
Best cost for generation 3 : 147

Generation number : 4 / 500
Best route for generation 4 : [2, 5, 0, 6, 1, 7, 4, 3]
Best cost for generation 4 : 149

Generation number : 5 / 500
Best route for generation 5 : [6, 3, 4, 2, 0, 5, 7, 1]
Best cost for generation 5 : 148

Generation number : 6 / 500
Best route for generation 6 : [6, 1, 0, 4, 7, 3, 5, 2]
Best cost for generation 6 : 146

Generation number : 7 / 500
Best route for generation 7 : [6, 2, 4, 3, 5, 0, 1, 7]
Best cost for generation 7 : 134

Generation number : 8 / 500
Best route for generation 8 : [1, 6, 2, 3, 5, 0, 4, 7]
Best cost for generation 8 : 137

Generation number : 9 / 500
Best route for generation 9 : [3, 6,

Best cost for generation 84 : 154

Generation number : 85 / 500
Best route for generation 85 : [5, 2, 6, 3, 4, 7, 1, 0]
Best cost for generation 85 : 145

Generation number : 86 / 500
Best route for generation 86 : [0, 1, 6, 5, 3, 2, 7, 4]
Best cost for generation 86 : 156

Generation number : 87 / 500
Best route for generation 87 : [2, 5, 6, 1, 7, 0, 3, 4]
Best cost for generation 87 : 166

Generation number : 88 / 500
Best route for generation 88 : [0, 1, 7, 2, 4, 6, 3, 5]
Best cost for generation 88 : 140

Generation number : 89 / 500
Best route for generation 89 : [5, 3, 6, 1, 0, 7, 2, 4]
Best cost for generation 89 : 149

Generation number : 90 / 500
Best route for generation 90 : [5, 3, 7, 1, 0, 6, 2, 4]
Best cost for generation 90 : 149

Generation number : 91 / 500
Best route for generation 91 : [2, 3, 5, 0, 1, 6, 7, 4]
Best cost for generation 91 : 140

Generation number : 92 / 500
Best route for generation 92 : [4, 0, 1, 6, 3, 2, 5, 7]
Best cost for generation 92 : 158

Gener

Best cost for generation 164 : 142

Generation number : 165 / 500
Best route for generation 165 : [4, 7, 1, 6, 2, 3, 5, 0]
Best cost for generation 165 : 137

Generation number : 166 / 500
Best route for generation 166 : [6, 1, 0, 5, 7, 3, 4, 2]
Best cost for generation 166 : 142

Generation number : 167 / 500
Best route for generation 167 : [1, 6, 4, 7, 2, 3, 5, 0]
Best cost for generation 167 : 143

Generation number : 168 / 500
Best route for generation 168 : [1, 6, 2, 4, 7, 3, 5, 0]
Best cost for generation 168 : 136

Generation number : 169 / 500
Best route for generation 169 : [1, 7, 6, 2, 4, 3, 5, 0]
Best cost for generation 169 : 134

Generation number : 170 / 500
Best route for generation 170 : [4, 3, 5, 0, 1, 6, 2, 7]
Best cost for generation 170 : 135

Generation number : 171 / 500
Best route for generation 171 : [6, 1, 7, 0, 5, 2, 3, 4]
Best cost for generation 171 : 149

Generation number : 172 / 500
Best route for generation 172 : [2, 7, 3, 5, 0, 1, 6, 4]
Best cost for ge

Best cost for generation 244 : 147

Generation number : 245 / 500
Best route for generation 245 : [7, 5, 3, 4, 0, 1, 6, 2]
Best cost for generation 245 : 142

Generation number : 246 / 500
Best route for generation 246 : [7, 4, 3, 5, 0, 1, 6, 2]
Best cost for generation 246 : 135

Generation number : 247 / 500
Best route for generation 247 : [5, 3, 4, 6, 1, 7, 2, 0]
Best cost for generation 247 : 143

Generation number : 248 / 500
Best route for generation 248 : [6, 3, 4, 2, 5, 7, 0, 1]
Best cost for generation 248 : 153

Generation number : 249 / 500
Best route for generation 249 : [3, 4, 7, 2, 6, 1, 0, 5]
Best cost for generation 249 : 135

Generation number : 250 / 500
Best route for generation 250 : [0, 1, 7, 4, 2, 6, 3, 5]
Best cost for generation 250 : 136

Generation number : 251 / 500
Best route for generation 251 : [0, 1, 7, 2, 4, 6, 3, 5]
Best cost for generation 251 : 140

Generation number : 252 / 500
Best route for generation 252 : [0, 7, 1, 6, 4, 2, 3, 5]
Best cost for ge

Best cost for generation 364 : 136

Generation number : 365 / 500
Best route for generation 365 : [7, 1, 0, 5, 3, 4, 2, 6]
Best cost for generation 365 : 134

Generation number : 366 / 500
Best route for generation 366 : [1, 7, 3, 5, 0, 4, 2, 6]
Best cost for generation 366 : 134

Generation number : 367 / 500
Best route for generation 367 : [4, 0, 5, 3, 2, 7, 1, 6]
Best cost for generation 367 : 141

Generation number : 368 / 500
Best route for generation 368 : [4, 3, 2, 5, 0, 7, 1, 6]
Best cost for generation 368 : 149

Generation number : 369 / 500
Best route for generation 369 : [1, 6, 2, 3, 4, 5, 7, 0]
Best cost for generation 369 : 156

Generation number : 370 / 500
Best route for generation 370 : [0, 5, 7, 1, 6, 4, 3, 2]
Best cost for generation 370 : 153

Generation number : 371 / 500
Best route for generation 371 : [7, 2, 0, 1, 6, 4, 3, 5]
Best cost for generation 371 : 152

Generation number : 372 / 500
Best route for generation 372 : [5, 3, 2, 4, 6, 7, 1, 0]
Best cost for ge

Best cost for generation 444 : 151

Generation number : 445 / 500
Best route for generation 445 : [6, 1, 7, 4, 2, 5, 3, 0]
Best cost for generation 445 : 154

Generation number : 446 / 500
Best route for generation 446 : [1, 0, 4, 2, 7, 5, 3, 6]
Best cost for generation 446 : 145

Generation number : 447 / 500
Best route for generation 447 : [0, 5, 3, 6, 1, 7, 4, 2]
Best cost for generation 447 : 142

Generation number : 448 / 500
Best route for generation 448 : [4, 7, 1, 6, 0, 5, 3, 2]
Best cost for generation 448 : 140

Generation number : 449 / 500
Best route for generation 449 : [1, 3, 5, 0, 4, 2, 6, 7]
Best cost for generation 449 : 154

Generation number : 450 / 500
Best route for generation 450 : [4, 0, 5, 3, 7, 1, 6, 2]
Best cost for generation 450 : 134

Generation number : 451 / 500
Best route for generation 451 : [6, 1, 7, 0, 5, 3, 4, 2]
Best cost for generation 451 : 132

Generation number : 452 / 500
Best route for generation 452 : [4, 3, 5, 7, 1, 6, 2, 0]
Best cost for ge

In [28]:
nodes=[0,1,2,3,4,5,6,7,8,9]
g = Graph(nodes)
for i in range(10):
    for j in range(10):
        if i>j:
            weight = np.random.randint(20)
            g.addEdge(i,j,weight)
            g.addEdge(j,i,weight)

generations=1500
path = g.gen_algo(0,generations) # executes the algorithm
total_cost = g.total_optimal_cost(path)
if total_cost:
    optimal_path(path,0)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Generation number : 1 / 1500
Best route for generation 1 : [3, 5, 7, 2, 0, 9, 8, 1, 6, 4]
Best cost for generation 1 : 63

Generation number : 2 / 1500
Best route for generation 2 : [3, 5, 7, 2, 0, 8, 9, 1, 6, 4]
Best cost for generation 2 : 65

Generation number : 3 / 1500
Best route for generation 3 : [0, 8, 9, 3, 2, 7, 5, 1, 6, 4]
Best cost for generation 3 : 58

Generation number : 4 / 1500
Best route for generation 4 : [1, 3, 2, 5, 0, 7, 8, 9, 6, 4]
Best cost for generation 4 : 60

Generation number : 5 / 1500
Best route for generation 5 : [1, 6, 2, 5, 0, 7, 8, 9, 3, 4]
Best cost for generation 5 : 59

Generation number : 6 / 1500
Best route for generation 6 : [0, 7, 1, 8, 9, 3, 6, 2, 4, 5]
Best cost for generation 6 : 69

Generation number : 7 / 1500
Best route for generation 7 : [0, 7, 1, 8, 9, 3, 6, 5, 4, 2]
Best cost for generation 7 : 61

Generation number : 8 / 1500
Best route for generation 8 : [3, 9, 6, 0, 8, 1, 4, 2, 5, 7]
Best cost for generation 8 : 50

Generation numbe

Best cost for generation 102 : 44

Generation number : 103 / 1500
Best route for generation 103 : [5, 2, 4, 6, 0, 7, 8, 9, 3, 1]
Best cost for generation 103 : 41

Generation number : 104 / 1500
Best route for generation 104 : [5, 2, 0, 6, 4, 7, 8, 9, 3, 1]
Best cost for generation 104 : 65

Generation number : 105 / 1500
Best route for generation 105 : [3, 5, 2, 4, 9, 8, 7, 0, 6, 1]
Best cost for generation 105 : 58

Generation number : 106 / 1500
Best route for generation 106 : [1, 7, 8, 9, 3, 4, 2, 5, 0, 6]
Best cost for generation 106 : 63

Generation number : 107 / 1500
Best route for generation 107 : [7, 0, 6, 4, 3, 2, 9, 5, 1, 8]
Best cost for generation 107 : 57

Generation number : 108 / 1500
Best route for generation 108 : [6, 4, 3, 9, 7, 0, 2, 5, 8, 1]
Best cost for generation 108 : 60

Generation number : 109 / 1500
Best route for generation 109 : [8, 1, 0, 2, 5, 7, 9, 4, 6, 3]
Best cost for generation 109 : 64

Generation number : 110 / 1500
Best route for generation 110 :


Generation number : 201 / 1500
Best route for generation 201 : [5, 1, 2, 0, 7, 8, 9, 3, 6, 4]
Best cost for generation 201 : 59

Generation number : 202 / 1500
Best route for generation 202 : [4, 9, 3, 2, 5, 0, 8, 7, 1, 6]
Best cost for generation 202 : 59

Generation number : 203 / 1500
Best route for generation 203 : [3, 9, 4, 2, 5, 0, 8, 7, 1, 6]
Best cost for generation 203 : 61

Generation number : 204 / 1500
Best route for generation 204 : [9, 7, 8, 0, 1, 6, 4, 5, 2, 3]
Best cost for generation 204 : 58

Generation number : 205 / 1500
Best route for generation 205 : [5, 7, 0, 2, 3, 9, 4, 6, 8, 1]
Best cost for generation 205 : 53

Generation number : 206 / 1500
Best route for generation 206 : [7, 5, 2, 0, 3, 9, 4, 6, 8, 1]
Best cost for generation 206 : 59

Generation number : 207 / 1500
Best route for generation 207 : [6, 4, 0, 7, 5, 3, 9, 8, 1, 2]
Best cost for generation 207 : 56

Generation number : 208 / 1500
Best route for generation 208 : [9, 5, 7, 0, 3, 8, 1, 6, 4, 2]
Be

Best cost for generation 301 : 35

Generation number : 302 / 1500
Best route for generation 302 : [2, 7, 0, 8, 1, 5, 9, 3, 4, 6]
Best cost for generation 302 : 49

Generation number : 303 / 1500
Best route for generation 303 : [2, 7, 8, 0, 1, 5, 9, 3, 4, 6]
Best cost for generation 303 : 56

Generation number : 304 / 1500
Best route for generation 304 : [6, 2, 7, 8, 0, 1, 5, 3, 9, 4]
Best cost for generation 304 : 61

Generation number : 305 / 1500
Best route for generation 305 : [6, 3, 7, 8, 0, 9, 2, 5, 1, 4]
Best cost for generation 305 : 68

Generation number : 306 / 1500
Best route for generation 306 : [8, 9, 5, 6, 3, 4, 0, 7, 2, 1]
Best cost for generation 306 : 72

Generation number : 307 / 1500
Best route for generation 307 : [7, 8, 9, 1, 6, 4, 2, 5, 0, 3]
Best cost for generation 307 : 70

Generation number : 308 / 1500
Best route for generation 308 : [7, 5, 3, 0, 6, 2, 9, 4, 1, 8]
Best cost for generation 308 : 72

Generation number : 309 / 1500
Best route for generation 309 :

Best cost for generation 381 : 52

Generation number : 382 / 1500
Best route for generation 382 : [2, 3, 4, 6, 1, 0, 8, 9, 7, 5]
Best cost for generation 382 : 51

Generation number : 383 / 1500
Best route for generation 383 : [6, 1, 5, 7, 0, 8, 3, 9, 2, 4]
Best cost for generation 383 : 50

Generation number : 384 / 1500
Best route for generation 384 : [7, 8, 0, 3, 9, 2, 4, 1, 6, 5]
Best cost for generation 384 : 53

Generation number : 385 / 1500
Best route for generation 385 : [3, 9, 5, 7, 8, 0, 2, 6, 1, 4]
Best cost for generation 385 : 56

Generation number : 386 / 1500
Best route for generation 386 : [7, 0, 6, 3, 2, 4, 9, 8, 1, 5]
Best cost for generation 386 : 51

Generation number : 387 / 1500
Best route for generation 387 : [7, 8, 1, 0, 3, 2, 9, 4, 6, 5]
Best cost for generation 387 : 56

Generation number : 388 / 1500
Best route for generation 388 : [2, 6, 9, 8, 0, 7, 5, 1, 4, 3]
Best cost for generation 388 : 58

Generation number : 389 / 1500
Best route for generation 389 :

...

Generation number : 1061 / 1500
Best route for generation 1061 : [5, 8, 7, 0, 2, 4, 6, 1, 9, 3]
Best cost for generation 1061 : 67

Generation number : 1062 / 1500
Best route for generation 1062 : [1, 6, 4, 0, 3, 9, 8, 7, 5, 2]
Best cost for generation 1062 : 54

Generation number : 1063 / 1500
Best route for generation 1063 : [2, 4, 6, 3, 1, 7, 8, 0, 9, 5]
Best cost for generation 1063 : 58

Generation number : 1064 / 1500
Best route for generation 1064 : [3, 9, 8, 5, 7, 0, 2, 4, 6, 1]
Best cost for generation 1064 : 53

Generation number : 1065 / 1500
Best route for generation 1065 : [2, 5, 1, 8, 0, 7, 6, 9, 3, 4]
Best cost for generation 1065 : 52

Generation number : 1066 / 1500
Best route for generation 1066 : [2, 7, 0, 8, 1, 3, 6, 4, 9, 5]
Best cost for generation 1066 : 50

Generation number : 1067 / 1500
Best route for generation 1067 : [6, 5, 7, 0, 8, 1, 3, 2, 4, 9]
Best cost for generation 1067 : 49

Generation number : 1068 / 1500
Best route for generation 1068 : [3, 9, 5, 2

Best cost for generation 1176 : 58

Generation number : 1177 / 1500
Best route for generation 1177 : [9, 6, 8, 0, 7, 5, 1, 4, 2, 3]
Best cost for generation 1177 : 56

Generation number : 1178 / 1500
Best route for generation 1178 : [1, 6, 9, 5, 7, 8, 0, 2, 3, 4]
Best cost for generation 1178 : 62

Generation number : 1179 / 1500
Best route for generation 1179 : [7, 2, 1, 3, 4, 6, 5, 9, 8, 0]
Best cost for generation 1179 : 58

Generation number : 1180 / 1500
Best route for generation 1180 : [2, 9, 3, 4, 6, 5, 1, 8, 0, 7]
Best cost for generation 1180 : 50

Generation number : 1181 / 1500
Best route for generation 1181 : [2, 4, 3, 6, 1, 5, 9, 8, 0, 7]
Best cost for generation 1181 : 58

Generation number : 1182 / 1500
Best route for generation 1182 : [1, 3, 2, 0, 7, 5, 4, 6, 9, 8]
Best cost for generation 1182 : 57

Generation number : 1183 / 1500
Best route for generation 1183 : [1, 0, 8, 7, 5, 9, 3, 4, 2, 6]
Best cost for generation 1183 : 48

Generation number : 1184 / 1500
Best rou

Best route for generation 1260 : [1, 8, 0, 9, 3, 2, 7, 5, 4, 6]
Best cost for generation 1260 : 57

Generation number : 1261 / 1500
Best route for generation 1261 : [7, 0, 1, 8, 2, 9, 3, 6, 4, 5]
Best cost for generation 1261 : 62

Generation number : 1262 / 1500
Best route for generation 1262 : [2, 4, 9, 8, 7, 0, 1, 3, 6, 5]
Best cost for generation 1262 : 52

Generation number : 1263 / 1500
Best route for generation 1263 : [2, 4, 1, 8, 0, 6, 3, 9, 7, 5]
Best cost for generation 1263 : 48

Generation number : 1264 / 1500
Best route for generation 1264 : [2, 4, 6, 0, 7, 5, 8, 1, 9, 3]
Best cost for generation 1264 : 55

Generation number : 1265 / 1500
Best route for generation 1265 : [6, 2, 5, 1, 8, 7, 0, 3, 9, 4]
Best cost for generation 1265 : 45

Generation number : 1266 / 1500
Best route for generation 1266 : [1, 8, 7, 0, 3, 9, 6, 4, 5, 2]
Best cost for generation 1266 : 52

Generation number : 1267 / 1500
Best route for generation 1267 : [2, 6, 4, 3, 9, 5, 1, 8, 7, 0]
Best cost fo

Generation number : 1381 / 1500
Best route for generation 1381 : [6, 2, 3, 9, 1, 8, 0, 7, 5, 4]
Best cost for generation 1381 : 52

Generation number : 1382 / 1500
Best route for generation 1382 : [5, 6, 3, 7, 0, 8, 1, 2, 4, 9]
Best cost for generation 1382 : 61

Generation number : 1383 / 1500
Best route for generation 1383 : [5, 6, 3, 0, 7, 8, 1, 2, 4, 9]
Best cost for generation 1383 : 58

Generation number : 1384 / 1500
Best route for generation 1384 : [4, 6, 3, 0, 7, 8, 1, 2, 5, 9]
Best cost for generation 1384 : 51

Generation number : 1385 / 1500
Best route for generation 1385 : [6, 3, 2, 5, 9, 0, 7, 8, 1, 4]
Best cost for generation 1385 : 52

Generation number : 1386 / 1500
Best route for generation 1386 : [6, 3, 9, 8, 0, 7, 2, 5, 1, 4]
Best cost for generation 1386 : 46

Generation number : 1387 / 1500
Best route for generation 1387 : [6, 5, 1, 7, 8, 0, 2, 4, 3, 9]
Best cost for generation 1387 : 61

Generation number : 1388 / 1500
Best route for generation 1388 : [2, 5, 9, 7

Best route for generation 1446 : [4, 9, 3, 6, 8, 0, 7, 2, 5, 1]
Best cost for generation 1446 : 64

Generation number : 1447 / 1500
Best route for generation 1447 : [1, 6, 4, 8, 0, 7, 5, 2, 3, 9]
Best cost for generation 1447 : 56

Generation number : 1448 / 1500
Best route for generation 1448 : [4, 1, 3, 9, 8, 7, 0, 6, 2, 5]
Best cost for generation 1448 : 62

Generation number : 1449 / 1500
Best route for generation 1449 : [0, 6, 5, 7, 3, 2, 4, 9, 8, 1]
Best cost for generation 1449 : 61

Generation number : 1450 / 1500
Best route for generation 1450 : [7, 0, 1, 8, 6, 4, 9, 3, 2, 5]
Best cost for generation 1450 : 45

Generation number : 1451 / 1500
Best route for generation 1451 : [7, 1, 0, 8, 6, 4, 9, 3, 2, 5]
Best cost for generation 1451 : 56

Generation number : 1452 / 1500
Best route for generation 1452 : [3, 9, 2, 5, 7, 0, 8, 4, 6, 1]
Best cost for generation 1452 : 55

Generation number : 1453 / 1500
Best route for generation 1453 : [7, 0, 1, 8, 6, 4, 9, 3, 2, 5]
Best cost fo

In [29]:
nodes=[]
for i in range(50):
    nodes.append(i)
g = Graph(nodes)
for i in range(50):
    for j in range(50):
        if i>j:
            weight = np.random.randint(20)
            g.addEdge(i,j,weight)
            g.addEdge(j,i,weight)
            
generations=500
path = g.gen_algo(0,generations) # executes the algorithm
total_cost = g.total_optimal_cost(path)
if total_cost:
    optimal_path(path,0)
    print("total_cost",total_cost)
else:
    print('Did not reach the goal!')

Generation number : 1 / 500
Best route for generation 1 : [27, 7, 46, 17, 6, 22, 8, 3, 39, 49, 48, 0, 9, 14, 32, 11, 16, 4, 43, 26, 40, 45, 10, 13, 23, 24, 38, 30, 35, 41, 44, 12, 33, 36, 25, 29, 1, 2, 34, 19, 28, 15, 47, 18, 5, 31, 42, 37, 20, 21]
Best cost for generation 1 : 414

Generation number : 2 / 500
Best route for generation 2 : [5, 18, 47, 20, 43, 48, 46, 49, 38, 37, 0, 41, 30, 8, 27, 44, 1, 15, 34, 28, 2, 45, 7, 12, 35, 13, 39, 22, 17, 11, 24, 42, 25, 3, 16, 23, 9, 26, 4, 6, 10, 33, 36, 31, 40, 32, 19, 21, 14, 29]
Best cost for generation 2 : 385

Generation number : 3 / 500
Best route for generation 3 : [27, 38, 15, 47, 44, 5, 18, 20, 43, 48, 46, 49, 37, 0, 41, 30, 8, 6, 34, 28, 2, 45, 7, 12, 35, 13, 39, 22, 17, 11, 24, 42, 25, 3, 16, 23, 9, 26, 4, 1, 10, 33, 36, 31, 40, 32, 19, 21, 14, 29]
Best cost for generation 3 : 375

Generation number : 4 / 500
Best route for generation 4 : [20, 43, 10, 5, 18, 1, 48, 46, 49, 38, 37, 0, 41, 30, 8, 27, 44, 47, 15, 34, 28, 2, 45, 7, 12

Best route for generation 30 : [4, 39, 28, 34, 9, 37, 43, 31, 45, 24, 7, 48, 44, 18, 14, 12, 15, 38, 26, 47, 32, 10, 6, 23, 3, 11, 49, 22, 46, 25, 5, 35, 19, 8, 20, 36, 41, 42, 30, 1, 21, 29, 27, 0, 33, 2, 17, 40, 16, 13]
Best cost for generation 30 : 334

Generation number : 31 / 500
Best route for generation 31 : [13, 14, 12, 15, 38, 26, 47, 32, 10, 6, 23, 4, 39, 28, 34, 9, 37, 43, 31, 45, 24, 7, 48, 44, 3, 11, 49, 22, 46, 25, 5, 35, 19, 8, 20, 36, 41, 42, 30, 1, 21, 29, 27, 0, 33, 2, 17, 40, 16, 18]
Best cost for generation 31 : 332

Generation number : 32 / 500
Best route for generation 32 : [21, 14, 40, 44, 48, 29, 12, 46, 47, 38, 22, 15, 39, 35, 25, 42, 30, 1, 5, 13, 17, 10, 37, 43, 28, 32, 26, 6, 23, 3, 11, 4, 9, 18, 36, 45, 31, 41, 2, 16, 34, 19, 0, 33, 24, 27, 7, 49, 8, 20]
Best cost for generation 32 : 307

Generation number : 33 / 500
Best route for generation 33 : [44, 48, 26, 37, 45, 28, 32, 10, 4, 9, 31, 43, 24, 18, 12, 38, 21, 14, 40, 29, 46, 47, 22, 15, 39, 35, 25, 42, 

Best route for generation 60 : [29, 26, 40, 15, 39, 35, 9, 1, 33, 28, 21, 17, 8, 46, 11, 36, 2, 47, 31, 24, 41, 16, 34, 30, 18, 6, 12, 45, 5, 38, 48, 14, 27, 20, 4, 10, 3, 37, 49, 42, 7, 44, 0, 19, 25, 32, 23, 13, 43, 22]
Best cost for generation 60 : 357

Generation number : 61 / 500
Best route for generation 61 : [14, 17, 16, 20, 35, 27, 34, 4, 10, 3, 44, 8, 0, 28, 39, 25, 31, 45, 49, 12, 11, 36, 2, 47, 24, 41, 30, 18, 6, 22, 38, 48, 29, 26, 40, 46, 5, 21, 15, 37, 19, 9, 1, 33, 7, 42, 32, 23, 13, 43]
Best cost for generation 61 : 346

Generation number : 62 / 500
Best route for generation 62 : [3, 22, 5, 46, 38, 15, 16, 23, 25, 43, 10, 36, 7, 4, 28, 41, 47, 2, 14, 17, 20, 35, 27, 34, 44, 8, 0, 39, 13, 31, 45, 49, 12, 11, 24, 30, 18, 6, 48, 29, 26, 40, 21, 37, 19, 9, 1, 33, 42, 32]
Best cost for generation 62 : 352

Generation number : 63 / 500
Best route for generation 63 : [0, 39, 37, 46, 44, 11, 29, 26, 25, 20, 8, 24, 36, 2, 47, 31, 41, 16, 34, 30, 6, 22, 38, 45, 48, 10, 12, 40, 42

Best route for generation 89 : [34, 49, 25, 15, 36, 0, 19, 33, 3, 23, 7, 4, 27, 17, 22, 45, 40, 20, 31, 32, 2, 6, 5, 37, 44, 16, 30, 42, 10, 1, 9, 29, 18, 11, 28, 47, 14, 41, 13, 35, 12, 8, 21, 43, 24, 39, 48, 46, 38, 26]
Best cost for generation 89 : 290

Generation number : 90 / 500
Best route for generation 90 : [1, 9, 29, 18, 11, 28, 47, 20, 45, 7, 49, 25, 48, 36, 40, 27, 6, 31, 23, 3, 4, 43, 39, 24, 34, 2, 17, 0, 19, 33, 5, 37, 44, 10, 30, 42, 22, 14, 41, 13, 35, 32, 15, 12, 8, 21, 46, 38, 16, 26]
Best cost for generation 90 : 314

Generation number : 91 / 500
Best route for generation 91 : [41, 16, 49, 25, 15, 36, 27, 7, 48, 29, 0, 19, 32, 3, 23, 5, 18, 1, 9, 11, 28, 47, 20, 45, 40, 6, 31, 4, 43, 39, 24, 34, 2, 17, 33, 37, 44, 10, 30, 42, 22, 14, 13, 35, 12, 8, 21, 46, 38, 26]
Best cost for generation 91 : 290

Generation number : 92 / 500
Best route for generation 92 : [34, 43, 36, 24, 39, 3, 46, 5, 18, 29, 7, 1, 27, 0, 19, 32, 23, 11, 31, 10, 48, 4, 44, 12, 47, 14, 21, 13, 35, 

Best route for generation 118 : [44, 46, 13, 15, 6, 31, 17, 43, 23, 9, 33, 19, 12, 26, 40, 45, 3, 22, 38, 7, 48, 42, 47, 18, 20, 29, 4, 32, 49, 37, 5, 11, 10, 36, 21, 8, 41, 25, 14, 28, 39, 24, 0, 30, 1, 35, 34, 16, 2, 27]
Best cost for generation 118 : 335

Generation number : 119 / 500
Best route for generation 119 : [1, 16, 15, 28, 36, 17, 41, 14, 39, 11, 10, 19, 32, 38, 12, 26, 40, 45, 3, 22, 43, 7, 23, 34, 29, 20, 18, 4, 13, 44, 46, 48, 6, 31, 9, 37, 5, 8, 0, 49, 24, 21, 25, 30, 35, 27, 33, 42, 47, 2]
Best cost for generation 119 : 318

Generation number : 120 / 500
Best route for generation 120 : [36, 21, 8, 41, 25, 14, 1, 16, 15, 28, 17, 39, 11, 38, 19, 32, 10, 12, 26, 40, 45, 3, 22, 43, 7, 23, 34, 29, 20, 18, 4, 13, 44, 46, 48, 6, 31, 9, 37, 5, 0, 49, 24, 30, 35, 27, 33, 42, 47, 2]
Best cost for generation 120 : 303

Generation number : 121 / 500
Best route for generation 121 : [36, 21, 8, 41, 25, 14, 1, 16, 15, 28, 17, 39, 11, 38, 19, 32, 10, 33, 26, 40, 45, 3, 22, 43, 7, 23, 

...

Best route for generation 354 : [5, 15, 33, 44, 3, 35, 0, 22, 34, 17, 31, 43, 40, 45, 18, 4, 32, 23, 28, 48, 19, 8, 26, 30, 7, 37, 42, 24, 14, 27, 21, 47, 11, 41, 12, 25, 46, 36, 13, 9, 1, 16, 39, 20, 49, 38, 6, 29, 10, 2]
Best cost for generation 354 : 331

Generation number : 355 / 500
Best route for generation 355 : [13, 25, 15, 33, 44, 3, 35, 0, 22, 34, 17, 31, 43, 40, 45, 18, 4, 32, 23, 28, 48, 19, 8, 26, 30, 7, 37, 42, 24, 14, 27, 21, 47, 11, 41, 12, 5, 46, 36, 9, 1, 16, 39, 20, 49, 38, 6, 29, 10, 2]
Best cost for generation 355 : 335

Generation number : 356 / 500
Best route for generation 356 : [23, 28, 48, 19, 22, 34, 17, 31, 43, 40, 45, 18, 4, 32, 8, 26, 30, 7, 37, 42, 5, 15, 49, 44, 3, 35, 0, 24, 14, 27, 21, 13, 11, 41, 12, 25, 46, 36, 47, 9, 1, 16, 39, 20, 33, 38, 6, 29, 10, 2]
Best cost for generation 356 : 344

Generation number : 357 / 500
Best route for generation 357 : [47, 11, 41, 12, 25, 46, 3, 35, 0, 22, 34, 17, 31, 4, 40, 45, 18, 43, 32, 23, 28, 48, 19, 8, 26, 30, 

Best route for generation 383 : [30, 4, 39, 2, 13, 41, 12, 40, 19, 43, 16, 0, 46, 42, 17, 11, 6, 29, 1, 18, 22, 14, 27, 8, 28, 3, 47, 44, 36, 33, 7, 45, 35, 23, 5, 15, 20, 21, 38, 34, 49, 25, 37, 26, 32, 24, 31, 48, 9, 10]
Best cost for generation 383 : 331

Generation number : 384 / 500
Best route for generation 384 : [19, 43, 16, 0, 46, 42, 17, 11, 6, 29, 1, 18, 22, 14, 27, 8, 28, 3, 30, 4, 39, 2, 13, 41, 12, 40, 47, 44, 36, 33, 7, 45, 35, 15, 5, 23, 20, 21, 38, 34, 49, 25, 37, 26, 32, 24, 31, 48, 9, 10]
Best cost for generation 384 : 334

Generation number : 385 / 500
Best route for generation 385 : [10, 3, 34, 49, 24, 32, 30, 4, 46, 20, 28, 41, 12, 40, 19, 37, 25, 9, 26, 31, 43, 16, 0, 6, 18, 22, 45, 47, 42, 8, 1, 14, 13, 48, 15, 44, 21, 38, 23, 11, 33, 7, 35, 5, 39, 36, 17, 2, 27, 29]
Best cost for generation 385 : 339

Generation number : 386 / 500
Best route for generation 386 : [30, 4, 46, 20, 13, 41, 12, 40, 19, 37, 25, 9, 32, 31, 43, 16, 0, 39, 42, 17, 11, 6, 29, 1, 18, 22, 1

Best route for generation 412 : [43, 13, 33, 22, 0, 34, 9, 3, 15, 26, 20, 29, 28, 23, 32, 38, 31, 17, 39, 46, 27, 8, 49, 37, 25, 6, 2, 7, 4, 12, 41, 21, 10, 11, 47, 16, 24, 30, 1, 14, 40, 35, 45, 48, 44, 18, 5, 42, 19, 36]
Best cost for generation 412 : 341

Generation number : 413 / 500
Best route for generation 413 : [24, 30, 0, 28, 21, 41, 14, 32, 15, 39, 27, 40, 26, 5, 37, 11, 7, 23, 20, 1, 46, 34, 29, 42, 6, 47, 16, 18, 49, 43, 13, 33, 22, 9, 3, 38, 31, 17, 8, 25, 2, 4, 12, 44, 48, 10, 45, 35, 19, 36]
Best cost for generation 413 : 324

Generation number : 414 / 500
Best route for generation 414 : [4, 12, 41, 21, 10, 11, 47, 16, 24, 30, 0, 28, 1, 32, 15, 39, 27, 40, 26, 5, 37, 7, 23, 20, 14, 46, 34, 29, 42, 6, 18, 49, 43, 13, 33, 22, 9, 3, 38, 31, 17, 8, 25, 2, 44, 48, 45, 35, 19, 36]
Best cost for generation 414 : 310

Generation number : 415 / 500
Best route for generation 415 : [21, 24, 30, 41, 14, 0, 15, 4, 12, 11, 10, 47, 16, 28, 1, 32, 39, 27, 40, 26, 5, 37, 7, 23, 20, 46, 3

Best route for generation 442 : [41, 5, 1, 32, 37, 16, 18, 38, 45, 49, 40, 35, 34, 27, 29, 12, 47, 2, 17, 36, 28, 7, 43, 30, 48, 9, 13, 6, 21, 14, 24, 33, 22, 19, 23, 31, 20, 26, 39, 10, 42, 8, 3, 4, 44, 11, 0, 46, 15, 25]
Best cost for generation 442 : 320

Generation number : 443 / 500
Best route for generation 443 : [43, 47, 41, 5, 1, 32, 37, 16, 18, 38, 45, 49, 40, 35, 34, 27, 29, 12, 2, 17, 36, 28, 7, 30, 48, 9, 13, 6, 21, 14, 11, 33, 22, 19, 23, 31, 20, 26, 39, 10, 42, 8, 3, 4, 44, 24, 0, 46, 15, 25]
Best cost for generation 443 : 345

Generation number : 444 / 500
Best route for generation 444 : [43, 47, 2, 5, 16, 18, 38, 45, 49, 40, 17, 28, 30, 48, 12, 6, 21, 14, 24, 15, 23, 31, 20, 26, 39, 10, 42, 8, 35, 34, 27, 29, 9, 46, 36, 33, 44, 1, 7, 22, 0, 3, 4, 25, 13, 11, 19, 41, 32, 37]
Best cost for generation 444 : 331

Generation number : 445 / 500
Best route for generation 445 : [41, 37, 43, 18, 2, 39, 20, 45, 49, 27, 47, 15, 28, 16, 38, 21, 23, 19, 5, 1, 32, 30, 12, 14, 26, 17,

Best route for generation 472 : [25, 47, 39, 38, 5, 45, 27, 28, 29, 11, 23, 20, 30, 21, 13, 33, 32, 19, 16, 46, 37, 6, 41, 17, 42, 24, 14, 0, 31, 48, 15, 7, 4, 49, 36, 35, 26, 40, 8, 44, 43, 10, 12, 3, 22, 9, 34, 2, 18, 1]
Best cost for generation 472 : 334

Generation number : 473 / 500
Best route for generation 473 : [6, 15, 37, 42, 23, 39, 38, 4, 27, 28, 32, 11, 25, 19, 33, 14, 0, 9, 34, 2, 48, 46, 41, 17, 45, 24, 49, 21, 13, 35, 3, 29, 16, 40, 43, 47, 8, 5, 7, 31, 36, 10, 12, 26, 44, 22, 18, 20, 30, 1]
Best cost for generation 473 : 337

Generation number : 474 / 500
Best route for generation 474 : [39, 38, 4, 27, 28, 32, 11, 25, 19, 33, 14, 0, 9, 34, 2, 48, 46, 41, 17, 45, 24, 49, 21, 13, 35, 3, 29, 16, 15, 43, 47, 8, 5, 7, 31, 36, 10, 12, 26, 44, 22, 20, 30, 18, 37, 6, 23, 40, 42, 1]
Best cost for generation 474 : 355

Generation number : 475 / 500
Best route for generation 475 : [9, 35, 26, 40, 8, 44, 43, 29, 7, 4, 49, 32, 36, 18, 27, 28, 23, 48, 5, 20, 31, 10, 12, 16, 3, 22, 6,